In [334]:
# !pip install selenium 
# 브라우저 컨트롤
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

from selenium import webdriver

driver = webdriver.Chrome()

In [335]:
browser = webdriver.Chrome('chromedriver')  # 크롬 브라우저로 연결
url = 'https://www.starbucks.co.kr/store/store_map.do'
browser.get(url)

In [336]:
# 지역검색 단추 클릭
reg_btn ='#container > div > form > fieldset > div > section > article.find_store_cont > article > header.loca_search > h3 > a'
browser.find_element_by_css_selector(reg_btn).click()

In [337]:
# 서울 단추 클릭
seoul_btn ='#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a'
browser.find_element_by_css_selector(seoul_btn).click()

In [339]:
# 전체 단추 클릭
all_btn = '#mCSB_2_container > ul > li:nth-child(1) > a'
browser.find_element_by_css_selector(all_btn).click()

In [340]:

############################################
# 데이터 읽어오기

html = browser.page_source
soup = BeautifulSoup(html, 'html.parser')
#soup

In [341]:
###################################
# <li> 리스트 아이템, or <tr>
starbucks_list = soup.select('li.quickResultLstCon')
starbucks_list

[<li class="quickResultLstCon" data-code="3758" data-hlytag="null" data-index="0" data-lat="37.6389955" data-long="127.3033633" data-name="남양주화도DT" data-storecd="1706" style="background:#fff"> <strong>남양주화도DT  <img alt="" class="setStoreFavBtn mCS_img_loaded" data-my_siren_order_store_yn="N" data-name="남양주화도DT" data-store="1706" data-yn="N" src="//image.istarbucks.co.kr/common/img/store/icon_fav_off.png"/></strong> <p class="result_details">경기도 남양주시 화도읍 수레로1137 주1동1~2층<br/>1522-3232</p> <i class="pin_generalDT">리저브 매장 2번</i></li>,
 <li class="quickResultLstCon" data-code="9797" data-hlytag="null" data-index="1" data-lat="37.6540359" data-long="127.24287989999993" data-name="평내호평역" data-storecd="610" style="background:#fff"> <strong>평내호평역  <img alt="" class="setStoreFavBtn mCS_img_loaded" data-my_siren_order_store_yn="N" data-name="평내호평역" data-store="610" data-yn="N" src="//image.istarbucks.co.kr/common/img/store/icon_fav_off.png"/></strong> <p class="result_details">경기도 남양주시 늘을1로16번길 2

In [342]:
starbucks_store = starbucks_list[0]
name = starbucks_store.select('strong')[0].text.strip() # 매장이름
lat =  starbucks_store['data-lat'].strip()  #위도 data-lat
lng =  starbucks_store['data-long'].strip() #경도 data-long
store_type = starbucks_store.select('i')[0]['class'][0][4:]    #매장타입
address = str(starbucks_store.select('p.result_details')[0]).split('<br />')[0].split('>')[1] #주소
tel = str(starbucks_store.select('p.result_details')[0]).split('<br/>')[1].split('<')[0]   #전화번호
              
print(name)
print(lat)
print(lng)
print(store_type)
print(address)
print(tel)

남양주화도DT
37.6389955
127.3033633
generalDT
경기도 남양주시 화도읍 수레로1137 주1동1~2층<br/
1522-3232


In [343]:
# q1 모든 지점 540매장의 모든 정보를 추출해서 데이터 프레임으로 만들어 x로 저장

# 매장명 위도 경도 매장타입 주소 전화번호

# seoul_starbucks_list.xlsx

In [344]:
starbucks_list_full = []

for item in starbucks_list:
    name = item.select('strong')[0].text.strip() # 매장이름
    lat =  item['data-lat'].strip()  #위도 data-lat
    lng =  item['data-long'].strip() #경도 data-long
    store_type = item.select('i')[0]['class'][0][4:]    #매장타입
    address = str(item.select('p.result_details')[0]).split('<br />')[0].split('>')[1] #주소
    tel = str(item.select('p.result_details')[0]).split('<br/>')[1].split('<')[0]   #전화번호
      
    starbucks_list_full.append([name, lat, lng, store_type, address, tel])


In [345]:
columns = ['매장명', '위도', '경도', '매장타입', '주소', '전화번호']
starbucks_list_df = pd.DataFrame(starbucks_list_full, columns = columns)
starbucks_list_df.to_excel('seoul_starbucks_list.xlsx', index = False)

In [346]:
sgg_names = []
for address in starbucks_list_df['주소']:
    sgg = address.split()[1]
    sgg_names.append(sgg)
sgg_names
starbucks_list_df['시군구명'] = sgg_names
starbucks_list_df.head()

,매장명,위도,경도,매장타입,주소,전화번호,시군구명
0,남양주화도DT,37.6389955,127.3033633,generalDT,경기도 남양주시 화도읍 수레로1137 주1동1~2층<br/,1522-3232,남양주시
1,평내호평역,37.6540359,127.24287989999993,general,"경기도 남양주시 늘을1로16번길 25, 성보빌딩 (호평동)<br/",1522-3232,남양주시
2,평내DT,37.6477605,127.23469380000006,generalDT,경기도 남양주시 경춘로 1269<br/,1522-3232,남양주시


In [347]:
seoul_sgg = pd.read_excel('6_Starbucks_Location/files/seoul_sgg_list.xlsx')

In [348]:
# 구별 스타벅스 매장의 수
starbucks_count = starbucks_list_df.pivot_table(index = '시군구명',
                             values = '매장명',
                              aggfunc = 'count').rename(columns={'매장명':'스타벅스 매장수'})

In [349]:
# 구별 주민등록인구수
seoul_sgg_pop = pd.read_excel('6_Starbucks_Location/files/sgg_pop.xlsx')
seoul_sgg_pop.head()

,시군구명,주민등록인구
0,종로구,"177,023"
1,중구,"138,102"
2,용산구,"250,844"
3,성동구,"316,113"
4,광진구,"386,292"


In [350]:
# starbucks_count seoul_sgg_pop 시군구명으로 합치기 seoul_sgg

In [351]:
seoul_sgg = pd.merge(seoul_sgg, seoul_sgg_pop, how='left', on='시군구명')
seoul_sgg.head()

,시군구코드,시군구명,위도,경도,주민등록인구
0,11320,도봉구,37.665861,127.031767,"371,095"
1,11380,은평구,37.617612,126.922700,"474,165"
2,11230,동대문구,37.583801,127.050700,"374,039"
3,11590,동작구,37.496504,126.944307,"407,966"
4,11545,금천구,37.460097,126.900155,"262,337"


In [352]:
seoul_sgg_biz = pd.read_excel('6_Starbucks_Location/files/sgg_biz.xlsx')
seoul_sgg_biz.head()

,시군구명,종사자수,사업체수
0,종로구,269106,40871
1,중구,423808,66190
2,용산구,133446,21178
3,성동구,162019,26130
4,광진구,123689,24531


In [353]:
seoul_sgg = pd.merge(seoul_sgg,
                    seoul_sgg_biz,
                    how = 'left',
                     on = '시군구명')
seoul_sgg

,시군구코드,시군구명,위도,경도,주민등록인구,종사자수,사업체수
0,11320,도봉구,37.665861,127.031767,"371,095",68669,18455
1,11380,은평구,37.617612,126.922700,"474,165",87693,24179
2,11230,동대문구,37.583801,127.050700,"374,039",143858,32994
3,11590,동작구,37.496504,126.944307,"407,966",103915,19609
4,11545,금천구,37.460097,126.900155,"262,337",223058,30080
5,11530,구로구,37.495486,126.858121,"449,187",210506,37445
6,11110,종로구,37.599100,126.986149,"177,023",269106,40871
7,11305,강북구,37.646995,127.014716,"342,669",69787,18654
8,11260,중랑구,37.595379,127.093967,"431,283",99241,27287
9,11680,강남구,37.495985,127.066409,"570,913",711278,73590


In [354]:
seoul_sgg.to_excel('seoul_sgg-stat.xlsx', index=False)

In [355]:
####################################################################
# 지도 그리기

! pip install folium

import pandas as pd
import folium
import json

In [356]:
seoul_starbucks = pd.read_excel('seoul_starbucks_list.xlsx')
seoul_starbucks

,매장명,위도,경도,매장타입,주소,전화번호
0,남양주화도DT,37.638996,127.303363,generalDT,경기도 남양주시 화도읍 수레로1137 주1동1~2층<br/,1522-3232
1,평내호평역,37.654036,127.242880,general,"경기도 남양주시 늘을1로16번길 25, 성보빌딩 (호평동)<br/",1522-3232
2,평내DT,37.647760,127.234694,generalDT,경기도 남양주시 경춘로 1269<br/,1522-3232


In [357]:
starbucks_map = folium.Map(location=[37.573050, 126.979189],
                          title = 'MAP',
                          zoom_start=11)
starbucks_map

In [358]:

# 역삼대로	37.501623	127.039375

lat = '37.501623'
lng = '127.039375'

folium.CircleMarker(location=[lat,lng],
                   fill = True,
                   fill_color = 'green',
                   fill_opacity = 1,
                   color = 'yellow',
                   weight = 1,
                   radius = 3).add_to(starbucks_map)

starbucks_map


In [359]:

for idx in seoul_starbucks.index:
    lat = seoul_starbucks.loc[idx, '위도']
    lng = seoul_starbucks.loc[idx, '경도']

folium.CircleMarker(location=[lat,lng],
                   fill = True,
                   fill_color = 'green',
                   fill_opacity = 1,
                   color = 'yellow',
                   weight = 1,
                   radius = 3).add_to(starbucks_map)
starbucks_map

In [360]:
# q3 매장 타입 별로 다른 색상으로

In [361]:
seoul_sgg_stat = pd.read_excel('6_Starbucks_Location/files/seoul_sgg_stat.xlsx')
seoul_sgg_stat

,시군구코드,시군구명,위도,경도,스타벅스_매장수,주민등록인구,종사자수_x,사업체수_x,종사자수_y,사업체수_y
0,11320,도봉구,37.665861,127.031767,2,"371,095",68669,18455,68669,18455
1,11380,은평구,37.617612,126.922700,8,"474,165",87693,24179,87693,24179
2,11230,동대문구,37.583801,127.050700,8,"374,039",143858,32994,143858,32994
3,11590,동작구,37.496504,126.944307,11,"407,966",103915,19609,103915,19609
4,11545,금천구,37.460097,126.900155,11,"262,337",223058,30080,223058,30080
5,11530,구로구,37.495486,126.858121,11,"449,187",210506,37445,210506,37445
6,11110,종로구,37.599100,126.986149,37,"177,023",269106,40871,269106,40871
7,11305,강북구,37.646995,127.014716,5,"342,669",69787,18654,69787,18654
8,11260,중랑구,37.595379,127.093967,7,"431,283",99241,27287,99241,27287
9,11680,강남구,37.495985,127.066409,81,"570,913",711278,73590,711278,73590


In [362]:
seoul_sgg_geo = json.load(open('6_Starbucks_Location/maps/seoul_sgg.geojson', encoding='utf-8'))
# 서울시 시군구 행정 경계 지도 데이터 파일 

In [363]:
seoul_sgg.geo['features'][1]

AttributeError: 'DataFrame' object has no attribute 'geo'

In [364]:
starbucks_bubble = folium.Map(location=[126.97621191600001, 126,97621191600001],
                              tiles = 'CartoDB positron',
                              zoom_start=11)
starbucks_bubble 

ValueError: Expected two (lat, lon) values for location, instead got: [126.97621191600001, 126, 97621191600001].

In [365]:
def style_function(feature):
    return {
        'opacity' : 0.7,
        'weight' : 1,
        'color' : 'color'
    }

In [ ]:
for idx in seoul_sgg_stat.index:
    lat = seoul_sgg_stat.loc[idx, '위도']
    lnd = seoul_sgg_stat.loc[idx, '경도']
    count = seoul_sgg_stat.loc[idx, '스타벅스_매장수']
    
    if count > starbucks_mean:
        fillColor = 'red'
    else
        fillColor = 'blue'
    folium.Circlemaker(location=[lat,lng],
                      color :' while',
                      fill_color = fillColor,
                      fillOpacity:0.7
                      dashArray:'5,5')